In [6]:
# Load dependencies
from keras.applications.vgg19 import VGG19
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Load the pre-trained VGG19 model

In [7]:
vgg19 = VGG19(include_top=False, # not getting the final dense layers
              weights='imagenet',
              input_shape=(224, 224, 3),
              pooling=None)

80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


## Freeze all the layers in the base VGGNet19 model

In [8]:
for layer in vgg19.layers:
    layer.trainable = False

## Add custom classification layers

In [10]:
# Instantiate the sequential model and add the VGG19 model
model = Sequential()
model.add(vgg19)
model.summary()

# Add the custom layers atop the VGG19 model
model.add(Flatten(name='flattened'))
model.add(Dropout(0.5, name='dropout'))
model.add(Dense(2, activation='softmax', name='predictions'))
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ vgg19 (Functional)                   │ (None, 7, 7, 512)           │      20,024,384 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 20,024,384 (76.39 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 20,024,384 (76.39 MB)

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ vgg19 (Functional)                   │ (None, 7, 7, 512)           │      20,024,384 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flattened (Flatten)                  │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ predictions (Dense)                  │ (None, 2)                   │          50,178 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 20,074,562 (76.58 MB)

 Trainable params: 50,178 (196.01 KB)

 Non-trainable params: 20,024,384 (76.39 MB)

## Compile the model for training

In [11]:
model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy'])

## Prepare the data for training

In [12]:
# # Getting the data
# !wget -c https://www.dropbox.com/s/86r9z1kb42422up/hot-dog-not-hot-dog.tar.gz
# !tar -xzf hot-dog-not-hot-dog.tar.gz

--2024-11-19 00:59:03--  https://www.dropbox.com/s/86r9z1kb42422up/hot-dog-not-hot-dog.tar.gz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.4.18, 2620:100:6019:18::a27d:412
Connecting to www.dropbox.com (www.dropbox.com)|162.125.4.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/86r9z1kb42422up/hot-dog-not-hot-dog.tar.gz [following]
--2024-11-19 00:59:03--  https://www.dropbox.com/s/raw/86r9z1kb42422up/hot-dog-not-hot-dog.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 404 Not Found
2024-11-19 00:59:03 ERROR 404: Not Found.

tar (child): hot-dog-not-hot-dog.tar.gz: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now


In [13]:
# Instantiate two image generator classes
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    data_format='channels_last',
    rotation_range=30,
    horizontal_flip=True,
    fill_mode='reflect'
)

valid_datagen = ImageDataGenerator(
    rescale=1.0/255,
    data_format='channels_last'
)

In [14]:
# Batch size
batch_size = 32

In [15]:
from os import SEEK_DATA
# Define the train and validation generators
train_generator = train_datagen.flow_from_directory(
    directory='./hot-dog-not-hot-dog/train',
    target_size=(224, 224),
    classes=['hot_dog', 'not_hot_dog'],
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=42
)

valid_generator = valid_datagen.flow_from_directory(
    directory='./hot-dog-not-hot-dog/test',
    target_size=(224, 224),
    classes=['hot_dog', 'not_hot_dog'],
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=42
)

Found 0 images belonging to 2 classes.
Found 0 images belonging to 2 classes.


In [ ]:
model.fit_generator(train_generator, steps_per_epoch=15,
                    epochs=16, validation_data=valid_generator,
                    validation_steps=15)